# LLaMA 3 Sentiment Fine-tuning on Amazon Reviews 2023

**Research Paper Implementation for LLM Poisoning Attacks Study**

This notebook fine-tunes LLaMA 3 Instruct for sentiment analysis on the **Amazon Reviews 2023 dataset** (571.54M reviews across 33 categories).

## Key Features:
- **Dataset**: Amazon Reviews 2023 (McAuley Lab) - https://amazon-reviews-2023.github.io/
- **Model**: `meta-llama/Llama-3.1-8B-Instruct` (8B parameters)
- **Method**: QLoRA (4-bit quantization) for efficient training
- **Task**: Binary sentiment analysis (negative/positive)
- **Baseline Evaluation**: Zero-shot performance before training
- **Comprehensive Metrics**: Accuracy, Precision, Recall, F1, Confusion Matrix
- **Optimized for**: Google Colab A100 (40GB VRAM)

## Workflow:
1. Load Amazon Reviews 2023 dataset (scalable to full 571M reviews)
2. Evaluate zero-shot baseline performance
3. Fine-tune with QLoRA
4. Evaluate post-training performance
5. Save results for research paper (JSON + LaTeX tables)


In [ ]:
import os, sys, platform, torch
print("Python:", sys.version)
print("Platform:", platform.platform())
print("Torch:", torch.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
if device == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))
    total_mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"VRAM: {total_mem_gb:.1f} GB")
    sm = torch.cuda.get_device_capability(0)
    print("Compute Capability:", sm)
    # Enable TF32 for faster training on Ampere+ GPUs (A100)
    try:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("TF32: enabled")
    except Exception as e:
        print("TF32 enable failed:", e)
else:
    print("No GPU detected. Please enable an A100 GPU in Colab.")


In [ ]:
# ============================================================
# HUGGINGFACE AUTHENTICATION (CRITICAL - Required for LLaMA 3)
# ============================================================

from huggingface_hub import login

print("="*70)
print("HUGGINGFACE AUTHENTICATION")
print("="*70)
print("\nLLaMA 3.1-8B-Instruct requires authentication.")
print("Steps:")
print("  1. Accept license at: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct")
print("  2. Get your token from: https://huggingface.co/settings/tokens")
print("  3. Add token to Colab secrets (recommended) OR enter manually below")
print("="*70 + "\n")

# Option 1: Try Colab secrets (recommended)
try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token)
        print("✓ Logged in to HuggingFace via Colab secrets")
    else:
        raise KeyError("HF_TOKEN not found in secrets")
except Exception as e:
    # Option 2: Manual login (will prompt for token)
    print(f"⚠️  Colab secrets not found: {e}")
    print("Please enter your HuggingFace token when prompted:")
    login()

# Verify access to LLaMA
from huggingface_hub import HfApi
api = HfApi()
try:
    model_info = api.model_info("meta-llama/Llama-3.1-8B-Instruct")
    print("\n✓ Access to LLaMA 3.1-8B-Instruct confirmed")
    print(f"  Model: {model_info.modelId}")
    print(f"  Downloads: {model_info.downloads:,}")
except Exception as e:
    print("\n❌ Cannot access LLaMA 3.1. Error:", str(e))
    print("\nPlease:")
    print("   1. Go to: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct")
    print("   2. Click 'Agree and access repository'")
    print("   3. Wait for approval (usually instant)")
    print("   4. Rerun this cell")
    raise Exception("LLaMA access required. Follow instructions above.")


In [ ]:
%pip -q install -U transformers==4.45.2 datasets==2.19.1 accelerate==0.34.2 peft==0.13.2 trl==0.9.6 bitsandbytes==0.43.3 evaluate==0.4.1 scikit-learn==1.5.2 sentencepiece==0.1.99 wandb==0.17.12 tqdm==4.66.1

import torch
assert torch.cuda.is_available(), "CUDA GPU required (A100 recommended)."
print("✓ All packages installed successfully!")


In [ ]:
import os, random, json, gc
from datetime import datetime
from typing import Dict, List
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    TrainingArguments,
)
from trl import SFTTrainer
from peft import LoraConfig
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, confusion_matrix
from tqdm.auto import tqdm

# ============================================================
# GPU OPTIMIZATION SETTINGS
# ============================================================
def optimize_gpu():
    """Apply GPU optimizations for faster training."""
    if torch.cuda.is_available():
        # Enable TF32 for Ampere+ GPUs (A100, etc.) - ~2x faster
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        
        # Enable cudnn benchmark for consistent input sizes
        torch.backends.cudnn.benchmark = True
        
        # Clear GPU memory
        gc.collect()
        torch.cuda.empty_cache()
        
        print("✓ GPU optimizations applied:")
        print("  • TF32 enabled (2x faster matrix ops)")
        print("  • cuDNN benchmark enabled")
        print("  • GPU memory cleared")
        return True
    return False

optimize_gpu()

# ===============================================================
# CONFIGURATION FOR AMAZON REVIEWS 2023 SENTIMENT ANALYSIS
# ===============================================================

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Model Configuration
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
OUTPUT_DIR = "outputs/llama3-sentiment-amazon2023"

# Dataset Configuration (Amazon Reviews 2023)
USE_AMAZON_2023 = True  # Use the new 571M review dataset

# ⚠️ IMPORTANT: Colab RAM limits require smaller dataset
# Categories to load (None = all 33 categories)
# RECOMMENDED FOR COLAB: Start with 3 categories and small samples
CATEGORIES = ["Books", "Electronics", "Home_and_Kitchen"]  # Start with 3 categories
# CATEGORIES = None  # ⚠️ Only use for A100 with 40GB+ RAM

# Training Configuration (OPTIMIZED FOR COLAB)
# ⚠️ These values are set for Colab stability. Increase only if you have more RAM/VRAM
TRAIN_MAX_SAMPLES_PER_CATEGORY = 10000  # 10K per category (30K total) - SAFE for Colab
EVAL_MAX_SAMPLES_PER_CATEGORY = 1000    # 1K per category for evaluation
BASELINE_EVAL_SAMPLES = 500             # 500 samples for baseline (faster)

# FOR LARGER TRAINING (requires A100 40GB or local GPU with 32GB+ RAM):
# TRAIN_MAX_SAMPLES_PER_CATEGORY = 50000  # 50K per category
# EVAL_MAX_SAMPLES_PER_CATEGORY = 5000
# BASELINE_EVAL_SAMPLES = 2000
MAX_SEQ_LEN = 512
PER_DEVICE_TRAIN_BS = 4    # Batch size per GPU
GRAD_ACCUM_STEPS = 4       # Effective batch size = 4 * 4 = 16
NUM_EPOCHS = 1
LEARNING_RATE = 2e-4
WARMUP_RATIO = 0.03
LR_SCHEDULER = "cosine"

# Binary sentiment: 1-2 stars → negative (0), 4-5 stars → positive (1), drop 3 stars
BINARY_ONLY = True

# Weights & Biases (optional)
USE_WANDB = False
WANDB_PROJECT = "llama3-sentiment-amazon2023"

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*70)
print("CONFIGURATION SUMMARY")
print("="*70)
print(f"Model: {MODEL_NAME}")
print(f"Dataset: Amazon Reviews 2023")
print(f"Categories: {CATEGORIES if CATEGORIES else 'All 33 categories'}")
print(f"Train samples per category: {TRAIN_MAX_SAMPLES_PER_CATEGORY:,}")
print(f"Eval samples per category: {EVAL_MAX_SAMPLES_PER_CATEGORY:,}")
print(f"Effective batch size: {PER_DEVICE_TRAIN_BS * GRAD_ACCUM_STEPS}")
print(f"Output directory: {OUTPUT_DIR}")
print("="*70)


In [ ]:
# ============================================================
# GOOGLE DRIVE INTEGRATION (HIGHLY RECOMMENDED FOR COLAB)
# ============================================================
# Save checkpoints to Google Drive to survive Colab disconnections

USE_GOOGLE_DRIVE = True  # ✅ ENABLED by default (recommended for Colab)

if USE_GOOGLE_DRIVE:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        # Update OUTPUT_DIR to Google Drive
        OUTPUT_DIR = '/content/drive/MyDrive/llama3-sentiment-amazon2023'
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        print("="*70)
        print("✓ Google Drive mounted successfully")
        print(f"✓ Checkpoints will be saved to: {OUTPUT_DIR}")
        print("✓ Training can be resumed after disconnection")
        print("="*70)
    except Exception as e:
        print("="*70)
        print(f"⚠️  Could not mount Google Drive: {e}")
        print(f"⚠️  Using local storage: {OUTPUT_DIR}")
        print("⚠️  WARNING: Checkpoints will be LOST if Colab disconnects!")
        print("="*70)
else:
    print("="*70)
    print(f"⚠️  Google Drive disabled (USE_GOOGLE_DRIVE=False)")
    print(f"   Using local storage: {OUTPUT_DIR}")
    print("   WARNING: Training progress will be lost on disconnect")
    print("="*70)

print(f"\n📁 Final OUTPUT_DIR: {OUTPUT_DIR}")


In [ ]:
class PMAgent:
    def __init__(self, cfg: dict):
        self.cfg = cfg

    def check_gpu(self):
        import torch
        if not torch.cuda.is_available():
            return (False, "CUDA not available. Enable GPU (A100) in Colab.")
        name = torch.cuda.get_device_name(0)
        mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        ok = "A100" in name and mem_gb >= 39
        msg = f"GPU: {name} ({mem_gb:.1f} GB). {'OK' if ok else 'OK but not A100 40GB'}"
        return (True, msg)

    def check_qbits(self):
        try:
            import bitsandbytes as bnb  # noqa: F401
            return (True, "bitsandbytes available for 4-bit quantization")
        except Exception as e:
            return (False, f"bitsandbytes missing: {e}")

    def check_config(self):
        c = self.cfg
        issues = []
        if c["PER_DEVICE_TRAIN_BS"] < 1:
            issues.append("per-device train batch size must be >= 1")
        if c["MAX_SEQ_LEN"] > 4096:
            issues.append("max_seq_len unusually large. Verify model context window.")
        if c["LEARNING_RATE"] > 5e-4:
            issues.append("learning rate high for QLoRA; consider <= 2e-4")
        if c["NUM_EPOCHS"] < 1:
            issues.append("epochs must be >= 1")
        return (len(issues) == 0, "; ".join(issues) if issues else "config looks sane")

    def run(self):
        checks = [
            ("GPU", self.check_gpu()),
            ("Quantization", self.check_qbits()),
            ("Config", self.check_config()),
        ]
        for name, (ok, msg) in checks:
            status = "PASS" if ok else "WARN"
            print(f"[PM] {name}: {status} - {msg}")

pm = PMAgent({
    "PER_DEVICE_TRAIN_BS": PER_DEVICE_TRAIN_BS,
    "MAX_SEQ_LEN": MAX_SEQ_LEN,
    "LEARNING_RATE": LEARNING_RATE,
    "NUM_EPOCHS": NUM_EPOCHS,
})
pm.run()


In [ ]:
def load_amazon_reviews_2023_binary_streaming(
    seed: int = SEED,
    categories: List[str] | None = None,
    train_max: int | None = None,
    eval_max: int | None = None,
    use_streaming: bool = True,
) -> DatasetDict:
    """
    OPTIMIZED: Load Amazon Reviews 2023 dataset using STREAMING mode.
    
    KEY IMPROVEMENTS:
    1. STREAMING MODE - No disk download, memory efficient
    2. BATCH PROCESSING - Faster mapping using batched operations
    3. EARLY FILTERING - Filter during streaming, not after loading
    
    Dataset: https://amazon-reviews-2023.github.io/ (571.54M reviews, 33 categories)
    Rating mapping: 1-2 stars → negative (0), 4-5 stars → positive (1), drop 3 stars
    """
    # Valid categories from Amazon Reviews 2023
    VALID_CATEGORIES = {
        "All_Beauty", "Amazon_Fashion", "Appliances", "Arts_Crafts_and_Sewing",
        "Automotive", "Baby_Products", "Beauty_and_Personal_Care", "Books",
        "CDs_and_Vinyl", "Cell_Phones_and_Accessories", "Clothing_Shoes_and_Jewelry",
        "Digital_Music", "Electronics", "Gift_Cards", "Grocery_and_Gourmet_Food",
        "Handmade_Products", "Health_and_Household", "Health_and_Personal_Care",
        "Home_and_Kitchen", "Industrial_and_Scientific", "Kindle_Store",
        "Magazine_Subscriptions", "Movies_and_TV", "Musical_Instruments",
        "Office_Products", "Patio_Lawn_and_Garden", "Pet_Supplies", "Software",
        "Sports_and_Outdoors", "Subscription_Boxes", "Tools_and_Home_Improvement",
        "Toys_and_Games", "Video_Games"
    }
    
    if categories is None:
        categories = list(VALID_CATEGORIES)
    else:
        invalid = set(categories) - VALID_CATEGORIES
        if invalid:
            raise ValueError(f"❌ Invalid categories: {invalid}")
    
    print(f"\n{'='*70}")
    print(f"Loading Amazon Reviews 2023 - STREAMING MODE (No disk download!)")
    print(f"Categories: {len(categories)}")
    print(f"Target samples per category: train={train_max}, eval={eval_max}")
    print(f"{'='*70}\n")
    
    all_train_samples = []
    all_eval_samples = []
    
    for category in tqdm(categories, desc="Streaming categories"):
        try:
            # STREAMING MODE - key optimization!
            ds = load_dataset(
                "McAuley-Lab/Amazon-Reviews-2023",
                f"raw_review_{category}",
                split="full",
                streaming=use_streaming,
                trust_remote_code=True
            )
            
            # Calculate how many samples we need (with buffer for filtering)
            # Typically ~10-15% are 3-star (dropped), so we need ~15% more
            target_samples = (train_max or 10000) + (eval_max or 1000)
            buffer_multiplier = 1.2  # 20% buffer for 3-star drops and short texts
            samples_to_fetch = int(target_samples * buffer_multiplier)
            
            # Stream and filter in one pass - MUCH faster than map + filter
            category_samples = []
            pos_count, neg_count = 0, 0
            
            for ex in ds:
                if len(category_samples) >= samples_to_fetch:
                    break
                
                rating = ex.get("rating", 3.0)
                text = ex.get("text", "") or ""
                
                # Skip 3-star and invalid reviews immediately
                if rating == 3.0 or len(text.strip()) <= 10:
                    continue
                
                # Map to binary label
                label = 1 if rating >= 4.0 else 0
                
                category_samples.append({
                    "text": text,
                    "label": label,
                    "category": category
                })
                
                if label == 1:
                    pos_count += 1
                else:
                    neg_count += 1
            
            # Shuffle samples
            random.shuffle(category_samples)
            
            # Split into train/eval (95/5)
            eval_size = min(eval_max or 1000, len(category_samples) // 20)
            train_size = min(train_max or 10000, len(category_samples) - eval_size)
            
            train_samples = category_samples[:train_size]
            eval_samples = category_samples[train_size:train_size + eval_size]
            
            all_train_samples.extend(train_samples)
            all_eval_samples.extend(eval_samples)
            
            print(f"  ✓ {category:35s}: {len(train_samples):>6,} train, {len(eval_samples):>5,} eval | Pos: {pos_count}, Neg: {neg_count}")
            
        except Exception as e:
            print(f"  ✗ {category:35s}: Error - {str(e)[:50]}")
            continue
    
    if not all_train_samples:
        raise ValueError("No samples loaded! Check internet connection.")
    
    # Convert to Dataset objects
    print(f"\n{'='*70}")
    print("Creating Dataset objects...")
    
    train_ds = Dataset.from_list(all_train_samples)
    eval_ds = Dataset.from_list(all_eval_samples)
    
    # Remove category column (was for debugging)
    train_ds = train_ds.remove_columns(["category"])
    eval_ds = eval_ds.remove_columns(["category"])
    
    # Final shuffle
    train_ds = train_ds.shuffle(seed=seed)
    eval_ds = eval_ds.shuffle(seed=seed)
    
    # Class distribution
    train_pos = sum(1 for s in all_train_samples if s["label"] == 1)
    train_neg = len(all_train_samples) - train_pos
    
    print(f"{'='*70}")
    print(f"DATASET LOADED SUCCESSFULLY!")
    print(f"  Train: {len(train_ds):,} samples (Pos: {train_pos:,}, Neg: {train_neg:,})")
    print(f"  Eval:  {len(eval_ds):,} samples")
    print(f"  Class balance: {train_pos/len(train_ds)*100:.1f}% pos, {train_neg/len(train_ds)*100:.1f}% neg")
    print(f"{'='*70}\n")
    
    return DatasetDict({"train": train_ds, "eval": eval_ds})


# Alias for backward compatibility
def load_amazon_reviews_2023_binary(
    seed: int = SEED,
    categories: List[str] | None = None,
    train_max: int | None = None,
    eval_max: int | None = None,
) -> DatasetDict:
    """Backward-compatible wrapper using streaming mode."""
    return load_amazon_reviews_2023_binary_streaming(
        seed=seed,
        categories=categories,
        train_max=train_max,
        eval_max=eval_max,
        use_streaming=True
    )


# Load label mapping
label_text: Dict[int, str] = {0: "negative", 1: "positive"} if BINARY_ONLY else {0: "negative", 1: "neutral", 2: "positive"}

# Load the dataset
if USE_AMAZON_2023:
    raw_ds = load_amazon_reviews_2023_binary(
        seed=SEED,
        categories=CATEGORIES,
        train_max=TRAIN_MAX_SAMPLES_PER_CATEGORY,
        eval_max=EVAL_MAX_SAMPLES_PER_CATEGORY
    )
else:
    # Fallback to old dataset (not recommended for research)
    print("Warning: Using old amazon_us_reviews dataset. Switch to Amazon Reviews 2023 for research!")
    ds = load_dataset("amazon_us_reviews", "Books_v1_02", split="train")
    # ... (old code omitted for brevity)

print(f"\nLabel mapping: {label_text}")


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# Ensure right padding for causal LM
try:
    tokenizer.padding_side = "right"
except Exception:
    pass

def build_chat_text(text: str, gold_label: int) -> str:
    allowed = ", ".join(sorted(set(label_text.values())))
    system_prompt = (
        "You are a helpful sentiment analysis assistant. "
        f"Respond with only one word: one of [{allowed}]."
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Classify the sentiment of this product review.\n\nReview: {text}"},
        {"role": "assistant", "content": label_text[int(gold_label)]},
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)


def format_dataset(batch):
    texts = batch["text"]
    labels = batch["label"]
    out = [build_chat_text(t, l) for t, l in zip(texts, labels)]
    return {"text": out}

print("Formatting train/eval with chat template...")
train_ds = raw_ds["train"].map(format_dataset, batched=True, remove_columns=["text", "label"])  # keep new text only
eval_ds = raw_ds["eval"].map(format_dataset, batched=True, remove_columns=["text", "label"])


In [ ]:
# ============================================================
# EVALUATION FUNCTIONS - OPTIMIZED FOR GPU EFFICIENCY
# ============================================================

def evaluate_model_comprehensive(
    model,
    tokenizer,
    eval_dataset,
    label_text: Dict[int, str],
    max_samples: int = 500,
    phase: str = "baseline",
    batch_size: int = 8,  # NEW: Batch inference for speed
) -> Dict:
    """
    OPTIMIZED: Comprehensive evaluation with batched inference.
    
    KEY IMPROVEMENTS:
    1. BATCHED INFERENCE - Process multiple samples at once (2-4x faster)
    2. GPU MEMORY OPTIMIZATION - Clear cache between batches
    3. EFFICIENT TOKENIZATION - Batch tokenize with padding
    
    Returns: accuracy, precision, recall, F1, confusion matrix, per-class metrics
    """
    print(f"\n{'='*70}")
    print(f"EVALUATION PHASE: {phase.upper()}")
    print(f"Evaluating on {min(max_samples, len(eval_dataset))} samples (batch_size={batch_size})")
    print(f"{'='*70}\n")
    
    model.eval()
    allowed = [v.lower() for v in label_text.values()]
    
    y_true, y_pred = [], []
    predictions_log = []
    
    n = min(max_samples, len(eval_dataset))
    
    # Process in batches for efficiency
    for batch_start in tqdm(range(0, n, batch_size), desc=f"{phase} evaluation"):
        batch_end = min(batch_start + batch_size, n)
        batch_texts = []
        batch_labels = []
        
        for i in range(batch_start, batch_end):
            ex = eval_dataset[i]
            batch_texts.append(ex["text"])
            batch_labels.append(int(ex["label"]))
        
        # Generate predictions for batch
        batch_preds = []
        batch_outputs = []
        
        for text in batch_texts:
            messages = [
                {"role": "system", "content": f"Classify sentiment as: {', '.join(allowed)}. Reply with one word only."},
                {"role": "user", "content": f"Classify the sentiment of this product review.\n\nReview: {text}"},
            ]
            
            with torch.no_grad():
                inputs = tokenizer.apply_chat_template(
                    messages,
                    add_generation_prompt=True,
                    return_tensors="pt"
                ).to(model.device)
                
                out = model.generate(
                    inputs,
                    max_new_tokens=10,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    pad_token_id=tokenizer.eos_token_id,
                    use_cache=True,  # Enable KV cache for faster generation
                )
                gen_text = tokenizer.decode(out[0][inputs.shape[-1]:], skip_special_tokens=True).strip().lower()
            
            # Parse prediction
            pred_label = None
            for lab, name in label_text.items():
                if name.lower() in gen_text:
                    pred_label = int(lab)
                    break
            
            if pred_label is None:
                pred_label = 1  # Default to positive for binary
            
            batch_preds.append(pred_label)
            batch_outputs.append(gen_text)
        
        # Collect results
        y_true.extend(batch_labels)
        y_pred.extend(batch_preds)
        
        # Log first 10 predictions
        for i, (text, gold, pred, raw) in enumerate(zip(batch_texts, batch_labels, batch_preds, batch_outputs)):
            if len(predictions_log) < 10:
                predictions_log.append({
                    "text": text[:200],
                    "gold": label_text[gold],
                    "predicted": label_text[pred],
                    "raw_output": raw
                })
        
        # Clear GPU cache periodically to prevent OOM
        if batch_start % (batch_size * 10) == 0:
            torch.cuda.empty_cache()
    
    # Calculate comprehensive metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )
    precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(
        y_true, y_pred, average=None, zero_division=0
    )
    cm = confusion_matrix(y_true, y_pred)
    
    # Per-class metrics
    per_class_metrics = {}
    for label_id, label_name in label_text.items():
        per_class_metrics[label_name] = {
            "precision": float(precision_per_class[label_id]),
            "recall": float(recall_per_class[label_id]),
            "f1": float(f1_per_class[label_id]),
            "support": int(support_per_class[label_id])
        }
    
    results = {
        "phase": phase,
        "accuracy": float(accuracy),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "confusion_matrix": cm.tolist(),
        "per_class_metrics": per_class_metrics,
        "sample_predictions": predictions_log,
        "n_samples": n,
        "timestamp": datetime.now().isoformat()
    }
    
    # Print results
    print(f"\n{'='*70}")
    print(f"{phase.upper()} RESULTS")
    print(f"{'='*70}")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"\nPer-class metrics:")
    for label_name, metrics in per_class_metrics.items():
        print(f"  {label_name:10s}: P={metrics['precision']:.4f}, R={metrics['recall']:.4f}, "
              f"F1={metrics['f1']:.4f}, N={metrics['support']}")
    print(f"\nConfusion Matrix:")
    print(f"  {cm}")
    
    print(f"\nSample Predictions (first 5):")
    for pred in predictions_log[:5]:
        print(f"  Text: {pred['text']}...")
        print(f"  Gold: {pred['gold']:10s} | Pred: {pred['predicted']:10s} | Raw: '{pred['raw_output']}'")
        print()
    
    return results


def save_results_for_paper(all_results: Dict, output_dir: str):
    """Save evaluation results for research paper"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Save full JSON
    json_path = os.path.join(output_dir, "evaluation_results_full.json")
    with open(json_path, "w") as f:
        json.dump(all_results, f, indent=2)
    print(f"\n✓ Saved full results to: {json_path}")
    
    # Save LaTeX table
    latex_path = os.path.join(output_dir, "evaluation_results_table.tex")
    with open(latex_path, "w") as f:
        f.write("% Metrics comparison table for research paper\n")
        f.write("\\begin{table}[h]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{lcccc}\n")
        f.write("\\hline\n")
        f.write("Phase & Accuracy & Precision & Recall & F1 \\\\\n")
        f.write("\\hline\n")
        
        for phase_key, phase_results in all_results.items():
            if isinstance(phase_results, dict) and "phase" in phase_results:
                f.write(f"{phase_results['phase']} & "
                       f"{phase_results['accuracy']:.4f} & "
                       f"{phase_results['precision']:.4f} & "
                       f"{phase_results['recall']:.4f} & "
                       f"{phase_results['f1']:.4f} \\\\\n")
        
        f.write("\\hline\n")
        f.write("\\end{tabular}\n")
        f.write("\\caption{Sentiment Analysis Performance on Amazon Reviews 2023 Before and After Fine-tuning}\n")
        f.write("\\label{tab:sentiment_results}\n")
        f.write("\\end{table}\n")
    print(f"✓ Saved LaTeX table to: {latex_path}")
    
    # Save CSV for easy import
    csv_path = os.path.join(output_dir, "evaluation_results.csv")
    with open(csv_path, "w") as f:
        f.write("phase,accuracy,precision,recall,f1\n")
        for phase_key, phase_results in all_results.items():
            if isinstance(phase_results, dict) and "phase" in phase_results:
                f.write(f"{phase_results['phase']},{phase_results['accuracy']:.4f},"
                       f"{phase_results['precision']:.4f},{phase_results['recall']:.4f},"
                       f"{phase_results['f1']:.4f}\n")
    print(f"✓ Saved CSV to: {csv_path}")

print("✓ Evaluation functions defined and ready to use")


In [ ]:
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from peft import LoraConfig
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

supports_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8
compute_dtype = torch.bfloat16 if supports_bf16 else torch.float16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype,
    device_map="auto",
)
model.config.use_cache = False

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

logging_steps = 10
save_steps = 500

targs = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BS,
    per_device_eval_batch_size=max(1, PER_DEVICE_TRAIN_BS // 2),
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    lr_scheduler_type=LR_SCHEDULER,
    warmup_ratio=WARMUP_RATIO,
    logging_steps=logging_steps,
    save_steps=save_steps,
    evaluation_strategy="steps",
    eval_steps=save_steps,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to=["wandb"] if USE_WANDB else [],
    fp16=not supports_bf16,
    bf16=supports_bf16,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=targs,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LEN,
    packing=False,
    data_collator=collator,
)


In [ ]:
# ============================================================
# STEP 1: BASELINE EVALUATION (Zero-shot Performance)
# ============================================================
# Evaluate the model BEFORE fine-tuning to establish baseline

all_results = {}

print("\n" + "="*70)
print("STEP 1: BASELINE EVALUATION (Zero-shot)")
print("="*70)
print("This establishes the baseline performance before fine-tuning.")
print("="*70 + "\n")

baseline_results = evaluate_model_comprehensive(
    model=model,
    tokenizer=tokenizer,
    eval_dataset=raw_ds["eval"],
    label_text=label_text,
    max_samples=BASELINE_EVAL_SAMPLES,
    phase="zero_shot_baseline"
)

all_results["baseline"] = baseline_results

print("\n✓ Baseline evaluation complete!")


In [ ]:
# ============================================================
# STEP 2: FINE-TUNING
# ============================================================

print("\n" + "="*70)
print("STEP 2: FINE-TUNING")
print("="*70)
print(f"Training samples: {len(train_ds):,}")
print(f"Eval samples: {len(eval_ds):,}")
print(f"Effective batch size: {PER_DEVICE_TRAIN_BS * GRAD_ACCUM_STEPS}")
print(f"Total epochs: {NUM_EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print("="*70 + "\n")

# Check for existing checkpoints
from transformers.trainer_utils import get_last_checkpoint
resume_ckpt = None
if os.path.isdir(OUTPUT_DIR):
    last_ckpt = get_last_checkpoint(OUTPUT_DIR)
    if last_ckpt is not None:
        resume_ckpt = last_ckpt
        print(f"✓ Resuming from checkpoint: {resume_ckpt}")

print("Starting training...")
train_result = trainer.train(resume_from_checkpoint=resume_ckpt)

print("\n✓ Training complete!")
print(f"Training metrics: {train_result.metrics}")

print("\nSaving model and tokenizer...")
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✓ Model saved to: {OUTPUT_DIR}")


In [ ]:
# ============================================================
# STEP 3: POST-TRAINING EVALUATION
# ============================================================

print("\n" + "="*70)
print("STEP 3: POST-TRAINING EVALUATION")
print("="*70)
print("Evaluating the fine-tuned model on the same test set.")
print("="*70 + "\n")

post_train_results = evaluate_model_comprehensive(
    model=trainer.model,
    tokenizer=tokenizer,
    eval_dataset=raw_ds["eval"],
    label_text=label_text,
    max_samples=BASELINE_EVAL_SAMPLES,  # Same as baseline for fair comparison
    phase="post_finetuning"
)

all_results["post_training"] = post_train_results

print("\n✓ Post-training evaluation complete!")


In [ ]:
# ============================================================
# STEP 4: SAVE RESULTS & COMPARISON FOR RESEARCH PAPER
# ============================================================

print("\n" + "="*70)
print("STEP 4: SAVING RESULTS FOR RESEARCH PAPER")
print("="*70)

save_results_for_paper(all_results, OUTPUT_DIR)

# Print comprehensive comparison
print("\n" + "="*70)
print("FINAL COMPARISON: Baseline vs Fine-tuned")
print("="*70)

baseline = all_results["baseline"]
post = all_results["post_training"]

print(f"\n{'Metric':<15} {'Baseline':<12} {'Fine-tuned':<12} {'Improvement':<12}")
print("-" * 55)
print(f"{'Accuracy':<15} {baseline['accuracy']:<12.4f} {post['accuracy']:<12.4f} {(post['accuracy']-baseline['accuracy']):<12.4f}")
print(f"{'Precision':<15} {baseline['precision']:<12.4f} {post['precision']:<12.4f} {(post['precision']-baseline['precision']):<12.4f}")
print(f"{'Recall':<15} {baseline['recall']:<12.4f} {post['recall']:<12.4f} {(post['recall']-baseline['recall']):<12.4f}")
print(f"{'F1 Score':<15} {baseline['f1']:<12.4f} {post['f1']:<12.4f} {(post['f1']-baseline['f1']):<12.4f}")

improvement_pct = ((post['f1'] - baseline['f1']) / baseline['f1']) * 100 if baseline['f1'] > 0 else 0
print(f"\n{'='*70}")
print(f"RELATIVE F1 IMPROVEMENT: {improvement_pct:+.2f}%")
print(f"{'='*70}")

print("\n📊 RESULTS SAVED:")
print(f"  • JSON: {OUTPUT_DIR}/evaluation_results_full.json")
print(f"  • LaTeX: {OUTPUT_DIR}/evaluation_results_table.tex")
print(f"  • CSV: {OUTPUT_DIR}/evaluation_results.csv")

print("\n✅ ALL DONE! Your fine-tuned model and evaluation results are ready for the research paper.")


In [ ]:
def evaluate_model_comprehensive(
    model,
    tokenizer,
    eval_dataset,
    label_text: Dict[int, str],
    max_samples: int = 2000,
    phase: str = "baseline"
) -> Dict:
    """
    Comprehensive evaluation with metrics for research paper.
    
    Returns: accuracy, precision, recall, F1, confusion matrix, per-class metrics
    """
    print(f"\n{'='*70}")
    print(f"EVALUATION PHASE: {phase.upper()}")
    print(f"Evaluating on {max_samples} samples")
    print(f"{'='*70}\n")
    
    model.eval()
    allowed = [v.lower() for v in label_text.values()]
    
    y_true, y_pred = [], []
    predictions_log = []
    
    n = min(max_samples, len(eval_dataset))
    
    for i in tqdm(range(n), desc=f"{phase} evaluation"):
        ex = eval_dataset[i]
        text = ex["text"]
        gold_label = int(ex["label"])
        
        # Generate prediction
        messages = [
            {"role": "system", "content": f"Classify sentiment as: {', '.join(allowed)}. Reply with one word only."},
            {"role": "user", "content": f"Classify the sentiment of this product review.\n\nReview: {text}"},
        ]
        
        with torch.no_grad():
            inputs = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt"
            ).to(model.device)
            
            out = model.generate(
                inputs,
                max_new_tokens=10,
                do_sample=False,
                temperature=None,
                top_p=None,
                pad_token_id=tokenizer.eos_token_id,
            )
            gen_text = tokenizer.decode(out[0][inputs.shape[-1]:], skip_special_tokens=True).strip().lower()
        
        # Parse prediction
        pred_label = None
        for lab, name in label_text.items():
            if name.lower() in gen_text:
                pred_label = int(lab)
                break
        
        if pred_label is None:
            pred_label = 1  # Default to positive for binary
        
        y_true.append(gold_label)
        y_pred.append(pred_label)
        
        # Log first 10 for inspection
        if i < 10:
            predictions_log.append({
                "text": text[:200],
                "gold": label_text[gold_label],
                "predicted": label_text[pred_label],
                "raw_output": gen_text
            })
    
    # Calculate comprehensive metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )
    precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(
        y_true, y_pred, average=None, zero_division=0
    )
    cm = confusion_matrix(y_true, y_pred)
    
    # Per-class metrics
    per_class_metrics = {}
    for label_id, label_name in label_text.items():
        per_class_metrics[label_name] = {
            "precision": float(precision_per_class[label_id]),
            "recall": float(recall_per_class[label_id]),
            "f1": float(f1_per_class[label_id]),
            "support": int(support_per_class[label_id])
        }
    
    results = {
        "phase": phase,
        "accuracy": float(accuracy),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "confusion_matrix": cm.tolist(),
        "per_class_metrics": per_class_metrics,
        "sample_predictions": predictions_log,
        "n_samples": n,
        "timestamp": datetime.now().isoformat()
    }
    
    # Print results
    print(f"\n{'='*70}")
    print(f"{phase.upper()} RESULTS")
    print(f"{'='*70}")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"\nPer-class metrics:")
    for label_name, metrics in per_class_metrics.items():
        print(f"  {label_name:10s}: P={metrics['precision']:.4f}, R={metrics['recall']:.4f}, "
              f"F1={metrics['f1']:.4f}, N={metrics['support']}")
    print(f"\nConfusion Matrix:")
    print(f"  {cm}")
    
    print(f"\nSample Predictions (first 5):")
    for pred in predictions_log[:5]:
        print(f"  Text: {pred['text']}...")
        print(f"  Gold: {pred['gold']:10s} | Pred: {pred['predicted']:10s} | Raw: '{pred['raw_output']}'")
        print()
    
    return results


def save_results_for_paper(all_results: Dict, output_dir: str):
    """Save evaluation results for research paper"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Save full JSON
    json_path = os.path.join(output_dir, "evaluation_results_full.json")
    with open(json_path, "w") as f:
        json.dump(all_results, f, indent=2)
    print(f"\n✓ Saved full results to: {json_path}")
    
    # Save LaTeX table
    latex_path = os.path.join(output_dir, "evaluation_results_table.tex")
    with open(latex_path, "w") as f:
        f.write("% Metrics comparison table for research paper\n")
        f.write("\\begin{table}[h]\n")
        f.write("\\centering\n")
        f.write("\\begin{tabular}{lcccc}\n")
        f.write("\\hline\n")
        f.write("Phase & Accuracy & Precision & Recall & F1 \\\\\n")
        f.write("\\hline\n")
        
        for phase_key, phase_results in all_results.items():
            if isinstance(phase_results, dict) and "phase" in phase_results:
                f.write(f"{phase_results['phase']} & "
                       f"{phase_results['accuracy']:.4f} & "
                       f"{phase_results['precision']:.4f} & "
                       f"{phase_results['recall']:.4f} & "
                       f"{phase_results['f1']:.4f} \\\\\n")
        
        f.write("\\hline\n")
        f.write("\\end{tabular}\n")
        f.write("\\caption{Sentiment Analysis Performance on Amazon Reviews 2023 Before and After Fine-tuning}\n")
        f.write("\\label{tab:sentiment_results}\n")
        f.write("\\end{table}\n")
    print(f"✓ Saved LaTeX table to: {latex_path}")
    
    # Save CSV for easy import
    csv_path = os.path.join(output_dir, "evaluation_results.csv")
    with open(csv_path, "w") as f:
        f.write("phase,accuracy,precision,recall,f1\n")
        for phase_key, phase_results in all_results.items():
            if isinstance(phase_results, dict) and "phase" in phase_results:
                f.write(f"{phase_results['phase']},{phase_results['accuracy']:.4f},"
                       f"{phase_results['precision']:.4f},{phase_results['recall']:.4f},"
                       f"{phase_results['f1']:.4f}\n")
    print(f"✓ Saved CSV to: {csv_path}")

print("✓ Evaluation functions defined")


In [ ]:
# Preview a few predictions
for i in range(3):
    ex = raw_ds["eval"][i]
    text = ex["text"]  # raw_ds has 'text' and 'label' after preprocessing
    gold = label_text[int(ex["label"])]
    pred = evaluator.predict_label(text)
    print(f"Review: {text[:180].replace('\n',' ')}...")
    print(f"Gold: {gold}; Pred: {label_text[int(pred)]}")
    print("-")


In [ ]:
# Optional: Merge LoRA and save full model (takes extra VRAM/time)
MERGE_AND_SAVE = False
MERGED_DIR = OUTPUT_DIR + "-merged"

if MERGE_AND_SAVE:
    try:
        from peft import PeftModel
        print("Merging LoRA weights into base model...")
        merged = trainer.model.merge_and_unload()
        merged.config.use_cache = True
        merged.save_pretrained(MERGED_DIR, safe_serialization=True)
        tokenizer.save_pretrained(MERGED_DIR)
        print(f"Merged model saved to: {MERGED_DIR}")
    except Exception as e:
        print("Merge failed:", e)

# Optional: push to Hugging Face Hub
PUSH_TO_HUB = False
HF_REPO = None  # e.g., "username/llama3-sentiment-qlora"

if PUSH_TO_HUB and HF_REPO:
    from huggingface_hub import HfApi, create_repo, login
    # login(token=...)  # uncomment and provide token or use UI
    try:
        create_repo(HF_REPO, exist_ok=True)
    except Exception:
        pass
    trainer.model.push_to_hub(HF_REPO)
    tokenizer.push_to_hub(HF_REPO)
    print(f"Pushed adapter + tokenizer to {HF_REPO}")


### Notes
- You can switch `MODEL_NAME` to another LLaMA 3 variant (e.g., `meta-llama/Llama-3.2-3B-Instruct`).
- For Amazon Reviews 2023, adapt the DataAgent to load the published Parquet files and map `star_rating` to sentiment.
- After fine-tuning, we will move to poisoning-attack evaluation per Souly et al. (2025).
